In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,53,2024-02-12,1,118.519231,98.200000,0.537808,9.969231,25.201923,0.251077,120.278846,...,11.660784,24.954902,0.181980,117.388235,0,2,0,2,23,2024
1,55,2024-02-12,1,124.055556,101.627778,0.585000,11.398148,24.000000,0.180778,122.085185,...,11.796154,22.176923,0.169192,110.682692,0,2,0,2,12,2024
2,54,2024-02-12,-1,111.471698,95.332075,0.533208,10.739623,24.522642,0.187566,115.222642,...,10.873585,27.649057,0.219623,119.777358,0,2,0,2,4,2024
3,54,2024-02-12,-7,111.981132,100.818868,0.528906,12.537736,22.660377,0.174189,110.754717,...,11.369811,25.096226,0.187981,115.450943,0,2,0,2,27,2024
4,54,2024-02-12,-2,114.981132,96.186792,0.542642,10.958491,29.062264,0.206792,119.645283,...,11.061538,24.117308,0.203654,114.307692,0,2,0,2,20,2024
5,54,2024-02-12,-1,114.603774,95.841509,0.559358,10.677358,26.115094,0.179491,119.724528,...,10.909434,22.030189,0.227000,121.115094,0,2,0,3,8,2024
6,54,2024-02-12,1,116.528302,97.896226,0.557604,11.311321,25.611321,0.213170,118.888679,...,12.450943,22.101887,0.180245,108.781132,0,2,0,2,19,2024
7,53,2024-02-12,-6,114.769231,102.409615,0.542231,11.761538,19.659615,0.179058,112.001923,...,10.620755,22.264151,0.204340,119.360377,0,2,0,2,30,2024
8,51,2024-02-12,4,119.040000,98.814000,0.556580,12.058000,27.488000,0.194960,119.076000,...,13.020755,29.239623,0.212981,117.762264,0,2,0,2,10,2024
9,53,2024-02-12,1,113.423077,96.761538,0.566481,12.978846,22.976923,0.220212,116.690385,...,11.086275,24.650980,0.217549,121.694118,0,2,0,4,18,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'FCNN':
        fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
        fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)
        
        model_file = fcnns.iloc[0]['File']
        
        model = torch.load('Models/'+model_file)
        scaler = joblib.load('Models/Scalers/FCNN.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'FCNN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,0
1,2024-02-12,IND,CHO,1.206186,4.700000,1
2,2024-02-12,CHI,ATL,2.420000,1.588235,0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0
4,2024-02-12,NYK,HOU,1.588235,2.420000,0
5,2024-02-12,DEN,MIL,1.847458,1.980392,0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1
7,2024-02-12,WAS,DAL,6.750000,1.117647,0
8,2024-02-12,GSW,UTA,1.892857,1.925926,1
9,2024-02-12,MIN,LAC,2.500000,1.555556,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,0
1,2024-02-12,IND,CHO,1.206186,4.700000,1
2,2024-02-12,CHI,ATL,2.420000,1.588235,0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0
4,2024-02-12,NYK,HOU,1.588235,2.420000,1
5,2024-02-12,DEN,MIL,1.847458,1.980392,0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1
7,2024-02-12,WAS,DAL,6.750000,1.117647,0
8,2024-02-12,GSW,UTA,1.892857,1.925926,0
9,2024-02-12,MIN,LAC,2.500000,1.555556,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,0
1,2024-02-12,IND,CHO,1.206186,4.700000,1
2,2024-02-12,CHI,ATL,2.420000,1.588235,0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0
4,2024-02-12,NYK,HOU,1.588235,2.420000,0
5,2024-02-12,DEN,MIL,1.847458,1.980392,0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1
7,2024-02-12,WAS,DAL,6.750000,1.117647,0
8,2024-02-12,GSW,UTA,1.892857,1.925926,0
9,2024-02-12,MIN,LAC,2.500000,1.555556,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,0
1,2024-02-12,IND,CHO,1.206186,4.700000,1
2,2024-02-12,CHI,ATL,2.420000,1.588235,0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0
4,2024-02-12,NYK,HOU,1.588235,2.420000,1
5,2024-02-12,DEN,MIL,1.847458,1.980392,0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1
7,2024-02-12,WAS,DAL,6.750000,1.117647,0
8,2024-02-12,GSW,UTA,1.892857,1.925926,0
9,2024-02-12,MIN,LAC,2.500000,1.555556,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,1.0
1,2024-02-12,IND,CHO,1.206186,4.700000,1.0
2,2024-02-12,CHI,ATL,2.420000,1.588235,0.0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0.0
4,2024-02-12,NYK,HOU,1.588235,2.420000,1.0
5,2024-02-12,DEN,MIL,1.847458,1.980392,0.0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1.0
7,2024-02-12,WAS,DAL,6.750000,1.117647,0.0
8,2024-02-12,GSW,UTA,1.892857,1.925926,1.0
9,2024-02-12,MIN,LAC,2.500000,1.555556,0.0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-12,PHI,CLE,4.700000,1.206186,1.0
1,2024-02-12,IND,CHO,1.206186,4.700000,1.0
2,2024-02-12,CHI,ATL,2.420000,1.588235,0.0
3,2024-02-12,SAS,TOR,2.850000,1.444444,0.0
4,2024-02-12,NYK,HOU,1.588235,2.420000,1.0
5,2024-02-12,DEN,MIL,1.847458,1.980392,0.0
6,2024-02-12,NOP,MEM,1.253165,4.100000,1.0
7,2024-02-12,WAS,DAL,6.750000,1.117647,0.0
8,2024-02-12,GSW,UTA,1.892857,1.925926,1.0
9,2024-02-12,MIN,LAC,2.500000,1.555556,0.0
